In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-02-24"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12727,2024-02-24,Eua Nba,00:00,Golden State Warriors,Charlotte Hornets,1.13,6.79,227.5,1.81,2.13,-12.5,1.89,18.00,O4deuqKJ,0.884956,0.147275,0.552486,0.469484,0.032231,0.6,0.6,NaN,NaN,275.728,81.544482,0.295742,460.876,152.639627,0.331195,229.188,693.908,204.53,701.72,0.0,1.0,0.0,0.0,1.010663,0.114860,1.145449,-0.10,-0.020,-6.500000,0.589372,0.5,-0.089372,6.90,1.380,4.195652,0.184962,0.2,0.015038
12728,2024-02-24,Eua Nba,00:00,Minnesota Timberwolves,Milwaukee Bucks,1.57,2.52,224.5,1.85,2.09,-4.5,1.97,3.25,jahCILC6,0.636943,0.396825,0.540541,0.478469,0.033768,0.6,0.2,NaN,NaN,214.376,71.446275,0.333276,182.414,58.502739,0.320714,143.880,185.602,146.52,128.70,0.0,0.0,0.0,0.0,0.328485,0.086145,0.346780,-2.55,-0.510,-1.117647,0.726943,0.6,-0.126943,-3.47,-0.694,-2.190202,0.671470,0.3,-0.371470
12729,2024-02-24,Eua Ncaa,00:00,San Jose State,Nevada,5.15,1.18,143.5,1.80,1.95,9.5,2.00,1.02,hGO8QgkH,0.194175,0.847458,0.555556,0.512821,0.041632,0.6,0.2,NaN,NaN,246.414,175.464425,0.712072,174.958,50.075961,0.286217,162.994,182.886,135.00,141.45,0.0,0.0,0.0,0.0,0.886955,0.056569,0.458917,-3.35,-0.670,-6.194030,0.500244,0.4,-0.100244,-0.21,-0.042,-4.285714,0.529011,0.6,0.070989
12730,2024-02-24,Eua Ncaa,18:00,UCF Knights,Texas Tech,1.90,1.90,137.5,1.80,1.91,-1.5,2.00,2.00,tvjSA6lb,0.526316,0.526316,0.555556,0.523560,0.052632,0.4,1.0,NaN,NaN,211.458,164.419279,0.777551,145.986,74.014725,0.506999,139.858,210.492,148.00,273.80,0.0,0.0,0.0,0.0,0.000000,0.041931,0.000000,2.50,0.500,1.800000,0.728380,0.8,0.071620,-1.23,-0.246,-3.658537,0.000000,0.0,0.000000
12731,2024-02-24,Lituânia Nkl,10:00,Vilkaviskis,Jurbarkas,1.64,2.17,162.5,1.80,1.86,-3.5,1.99,2.41,4EOrtknk,0.609756,0.460829,0.555556,0.537634,0.070586,0.0,1.0,NaN,NaN,231.652,72.127764,0.311363,234.638,71.961405,0.306691,233.458,237.630,148.19,205.80,0.0,0.0,0.0,0.0,0.196728,0.023184,0.134993,7.19,1.438,0.445063,0.000000,0.0,0.000000,-3.12,-0.624,-1.875000,0.485908,0.3,-0.185908
12732,2024-02-24,Lituânia Nkl,11:00,Telsiai,Suduva-Mantinga,1.69,2.06,154.5,1.87,1.79,-2.5,1.94,2.50,GlSnu91e,0.591716,0.485437,0.534759,0.558659,0.077153,0.6,0.6,NaN,NaN,149.340,49.215148,0.329551,105.782,17.034199,0.161031,110.364,116.796,107.90,96.38,0.0,0.0,0.0,0.0,0.139536,0.030912,0.178369,-1.39,-0.278,-2.482014,0.741686,0.8,0.058314,-1.46,-0.292,-3.630137,0.737071,0.7,-0.037071
12733,2024-02-24,Lituânia Nkl,11:00,Klaipedos Neptunas,Ukmerges Stekas,2.06,1.69,155.5,1.85,1.85,1.5,1.91,1.56,zFCsKPoi,0.485437,0.591716,0.540541,0.540541,0.077153,0.6,0.0,NaN,NaN,175.666,44.990013,0.256111,108.194,4.109700,0.037985,146.822,0.000,122.82,109.85,0.0,0.0,0.0,0.0,0.139536,0.000000,0.142644,2.49,0.498,2.128514,0.417015,0.5,0.082985,0.00,0.000,inf,0.000000,0.0,0.000000
12734,2024-02-24,Lituânia Moteru Lyga Feminina,14:00,Kibirkstis Vilnius F,LCC F,1.99,1.74,142.5,1.85,1.85,1.5,1.85,1.60,lCskZMbD,0.502513,0.574713,0.540541,0.540541,0.077225,0.0,0.0,NaN,NaN,108.558,66.653128,0.613986,139.242,130.584911,0.937827,0.000,0.000,109.18,80.58,0.0,0.0,0.0,0.0,0.094786,0.000000,0.102479,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12735,2024-02-24,Taiwan Liga T1,03:00,Taipei Taishin Mars,Tainan TSG GhostHawks,1.74,1.99,206.5,1.85,1.85,-2.5,2.01,2.41,jmhSZkFD,0.574713,0.502513,0.540541,0.540541,0.077225,0.8,0.6,NaN,NaN,311.822,64.199807,0.205886,243.712,101.010976,0.414469,297.756,236.990,230.05,384.56,0.0,0.0,0.0,0.0,0.094786,0.000000,

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Golden State Warriors,Charlotte Hornets,227.5,1.81,1.0000,Over
1,00:00,Eua Nba,Minnesota Timberwolves,Milwaukee Bucks,224.5,1.85,1.0000,Over
2,18:00,Eua Ncaa,UCF Knights,Texas Tech,137.5,1.80,0.9932,Over
3,10:00,Lituânia Nkl,Vilkaviskis,Jurbarkas,162.5,1.80,1.0000,Over
4,11:00,Lituânia Nkl,Telsiai,Suduva-Mantinga,154.5,1.87,1.0000,Over
5,03:00,Taiwan Liga T1,Taipei Taishin Mars,Tainan TSG GhostHawks,206.5,1.85,1.0000,Over
6,09:00,Turquia Tb2L,Uskudar,High Touch,142.5,1.82,1.0000,Over
7,14:00,Eua Ncaa,Northeastern,Hampton,144.5,1.91,1.0000,Over
8,16:00,Eua Ncaa,Ball State,Eastern Michigan,143.5,1.91,1.0000,Over
9,16:00,Eua Ncaa,Le Moyne,Fairleigh Dickinson,157.5,1.80,1.0000,Over
